In [27]:
import  pandas  as  pd
import os
import requests
import matplotlib.pyplot as plt
import geopandas as gpd
import pyproj
import datetime
from geopandas import GeoDataFrame as gdf
from shapely.geometry import Point
from shapely.geometry import Polygon
import json
import sys
sys.path.append('/Users/alenastern/Documents/Spring2018/Machine_Learning/evictions-learn/src/data/census')
import scraper as sp
%matplotlib inline

In [20]:
state_list = ['18', '17', '26', '39', '55', '10', '11', '12', '13', '24', '37', '45', '51', '54']
df = pd.read_table('national_county.txt', sep = ',', converters={'01': lambda x: str(x), '001': lambda x: str(x)})
df = df[df['01'].isin(state_list)]

In [21]:
df['county'] = df['01'] + df['001']

In [50]:
len(df)

1026

In [22]:
acs_results_2016 = requests.get('https://api.census.gov/data/2016/acs/acs5?get=NAME,B15002_001E,B15003_022E,B15003_023E,B15003_024E,B15003_025E,B25010_001E&for=block%20group:*&in=state:10%20county:001&key=91aa6a44edb890abeea862f2d9befb5d520a5cc4')
data = acs_results_2016.text
total_df = pd.read_json(data)
errors = []


# Query ACS Data from Census API
for i in range(1, len(df)):
    acs_results_2016 = requests.get('https://api.census.gov/data/2016/acs/acs5?get=NAME,B15002_001E,B15003_022E,B15003_023E,B15003_024E,B15003_025E,B25010_001E&for=block%20group:*&in=state:{}%20county:{}&key=91aa6a44edb890abeea862f2d9befb5d520a5cc4'.format(df.iloc[i,1], df.iloc[i,2]))
    print(df.iloc[i,1], df.iloc[i,2])
    if acs_results_2016.status_code != 204:
        data = acs_results_2016.text
        acs_df = pd.read_json(data)
        total_df = pd.concat([total_df, acs_df])
    else:
        errors.append([df.iloc[i,1], df.iloc[i,2]])



10 003
10 005
11 001
12 001
12 003
12 005
12 007
12 009
12 011
12 013
12 015
12 017
12 019
12 021
12 023
12 027
12 029
12 031
12 033
12 035
12 037
12 039
12 041
12 043
12 045
12 047
12 049
12 051
12 053
12 055
12 057
12 059
12 061
12 063
12 065
12 067
12 069
12 071
12 073
12 075
12 077
12 079
12 081
12 083
12 085
12 086
12 087
12 089
12 091
12 093
12 095
12 097
12 099
12 101
12 103
12 105
12 107
12 109
12 111
12 113
12 115
12 117
12 119
12 121
12 123
12 125
12 127
12 129
12 131
12 133
13 001
13 003
13 005
13 007
13 009
13 011
13 013
13 015
13 017
13 019
13 021
13 023
13 025
13 027
13 029
13 031
13 033
13 035
13 037
13 039
13 043
13 045
13 047
13 049
13 051
13 053
13 055
13 057
13 059
13 061
13 063
13 065
13 067
13 069
13 071
13 073
13 075
13 077
13 079
13 081
13 083
13 085
13 087
13 089
13 091
13 093
13 095
13 097
13 099
13 101
13 103
13 105
13 107
13 109
13 111
13 113
13 115
13 117
13 119
13 121
13 123
13 125
13 127
13 129
13 131
13 133
13 135
13 137
13 139
13 141
13 143
13 145
13 147

In [23]:
errors

[['51', '515']]

# Scraping Building Permit Data

In [28]:
ul = sp.scrape_links('https://www2.census.gov/econ/bps/County/', [])

In [29]:
permits = sp.url_to_pd(ul)

In [30]:
permits.columns

Index(['Survey', 'FIPS', 'FIPS.1', 'Region', 'Division', 'County',
       'Unnamed: 6', '1-unit', 'Unnamed: 8', 'Unnamed: 9', '2-units',
       'Unnamed: 11', 'Unnamed: 12', '3-4 units', 'Unnamed: 14', 'Unnamed: 15',
       '5+ units', 'Unnamed: 17', 'Unnamed: 18', '1-unit rep', 'Unnamed: 20',
       'Unnamed: 21', '2-units rep', 'Unnamed: 23', 'Unnamed: 24',
       '3-4 units rep', 'Unnamed: 26', 'Unnamed: 27', ' 5+units rep'],
      dtype='object')

In [39]:
permits.iloc[4]

index                                      2000
Survey                                       01
FIPS                                        007
FIPS.1                                        3
Region                                        6
Division         Bibb County                   
County                                        4
Unnamed: 6                                    4
1-unit                                   310000
Unnamed: 8                                    0
Unnamed: 9                                    0
2-units                                       0
Unnamed: 11                                   0
Unnamed: 12                                   0
3-4 units                                     0
Unnamed: 14                                   0
Unnamed: 15                                   0
5+ units                                      0
Unnamed: 17                                   4
Unnamed: 18                                   4
1-unit rep                              

In [41]:
permits = permits.reset_index()
permits.head()

ValueError: cannot insert level_0, already exists

In [51]:
permits.rename(index=str, columns={"index": "year", "Survey": "state", "FIPS": "county", "FIPS.1": "region",
                             "Region": "division", "Division": "name", "County": "bldg_1", "Unnamed: 6": "units_1",
                             "1-unit": "value_1", "Unnamed: 8": "bldg_2", "Unnamed: 9": "units_2", "2-units": "value_2",
                              "Unnamed: 11": "bldg_3", "Unnamed: 12": "units_3", "3-4 units": "value_3-4",
                              "Unnamed: 14": "bldg_5", "Unnamed: 15": "units_5", "5+ units": "value_5"}, inplace = True)

In [44]:
permits.drop(['Unnamed: 17', 'Unnamed: 18', '1-unit rep', 'Unnamed: 20',
       'Unnamed: 21', '2-units rep', 'Unnamed: 23', 'Unnamed: 24',
       '3-4 units rep', 'Unnamed: 26', 'Unnamed: 27', ' 5+units rep'], axis=1, inplace = True)

In [56]:
permits.drop('0', axis = 0, inplace = True)

In [74]:
permits.head()

,year,state,county,region,division,name,bldg_1,units_1,value_1,bldg_2,...,value_2,bldg_3,units_3,value_3-4,bldg_5,units_5,value_5,total_bldg,total_units,total_value
1,2000,01,001,3,6,Autauga County,181,181,9854805,0,...,0,0,0,0,0,0,0,181000,181000,9854805000
2,2000,01,003,3,6,Baldwin County,1841,1841,237144524,12,...,1818631,0,0,0,10,607,71423749,184112010,1841240607,2371445241818631071423749
3,2000,01,005,3,6,Barbour County,27,27,2721322,0,...,0,0,0,0,1,40,1514892,27001,270040,2721322001514892
4,2000,01,007,3,6,Bibb County,4,4,310000,0,...,0,0,0,0,0,0,0,4000,4000,310000000
5,2000,01,009,3,6,Blount County,59,59,8547016,0,...,0,0,0,0,1,12,523800,59001,590012,854701600523800


In [77]:
cols = ['bldg_1', 'units_1', 'value_1', 'bldg_2', 'units_2', 'value_2', 'bldg_3', 'units_3', 'value_3-4',
       'bldg_5', 'units_5', 'value_5']

permits[cols] = permits[cols].apply(pd.to_numeric, errors='coerce')


permits['total_bldg'] = permits['bldg_1'] + permits['bldg_2'] + permits['bldg_3'] + permits['bldg_5'] 
permits['total_units'] = permits['units_1'] + permits['units_2'] + permits['units_3'] + permits['units_5']
permits['total_value'] = permits['value_1'] + permits['value_2'] + permits['value_3-4'] + permits['value_5']

In [78]:
permits_div = permits[(permits['division'] == '3') | (permits['division'] == '5')]

In [79]:
permits_div.iloc[4]

year                                     2000
state                                      12
county                                    001
region                                      3
division                                    5
name           Alachua County                
bldg_1                                   1072
units_1                                  1072
value_1                           1.13188e+08
bldg_2                                      5
units_2                                    10
value_2                                287970
bldg_3                                     15
units_3                                    60
value_3-4                          4.3782e+06
bldg_5                                     65
units_5                                   831
value_5                           4.67198e+07
total_bldg                               1157
total_units                              1973
total_value                       1.64574e+08
Name: 309, dtype: object

In [80]:
permits_total = permits_div[['year', 'state', 'county', 'region', 'division', 'total_bldg', 'total_units', 'total_value']]

In [82]:
permits_total.head()

,year,state,county,region,division,total_bldg,total_units,total_value
305,2000,10,001,3,5,854.0,858.0,75130874.0
306,2000,10,003,3,5,1699.0,2331.0,164781826.0
307,2000,10,005,3,5,1422.0,1422.0,174176014.0
308,2000,11,001,3,5,201.0,806.0,53992642.0
309,2000,12,001,3,5,1157.0,1973.0,164573829.0


In [83]:
permits_total.to_csv('permits.csv', index = False)

In [ ]:

# Clean ACS data frame and rename columns# Clean  
variables = {'B15002_001E': "total_pop_over25", 'B15003_022E': "bachelors", 'B15003_023E': "masters", 'B15003_024E': "professional", 'B15003_024E': "doctorate"
            'B25010_001E': "avg_hh_size"}
acs_df.columns = acs_df.iloc[0]
acs_df.drop([0], inplace = True)
acs_df.rename(columns = variables, inplace=True)